In [1]:
import findspark
findspark.init('/home/mojtaba/spark-2.1.0-bin-hadoop2.7')
import os
data_dir = os.path.join(os.getcwd(), 'data')

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('ops').getOrCreate()

In [4]:
df = spark.read.csv(os.path.join(data_dir, 'appl_stock.csv'),
                    inferSchema=True,
                    header=True)

In [5]:
df.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



In [6]:
df.head(3)[0]

Row(Date=datetime.datetime(2010, 1, 4, 0, 0), Open=213.429998, High=214.499996, Low=212.38000099999996, Close=214.009998, Volume=123432400, Adj Close=27.727039)

## Using sql

If you already now sql, you can use the syntax, for example for filtering:

In [7]:
df.filter("Close < 500").show()

+--------------------+------------------+------------------+------------------+------------------+---------+------------------+
|                Date|              Open|              High|               Low|             Close|   Volume|         Adj Close|
+--------------------+------------------+------------------+------------------+------------------+---------+------------------+
|2010-01-04 00:00:...|        213.429998|        214.499996|212.38000099999996|        214.009998|123432400|         27.727039|
|2010-01-05 00:00:...|        214.599998|        215.589994|        213.249994|        214.379993|150476200|27.774976000000002|
|2010-01-06 00:00:...|        214.379993|            215.23|        210.750004|        210.969995|138040000|27.333178000000004|
|2010-01-07 00:00:...|            211.75|        212.000006|        209.050005|            210.58|119282800|          27.28265|
|2010-01-08 00:00:...|        210.299994|        212.000006|209.06000500000002|211.98000499999998|111902

Let's see you only want to see the `Open` column:

In [8]:
df.filter("Close < 500").select('Open').show()

+------------------+
|              Open|
+------------------+
|        213.429998|
|        214.599998|
|        214.379993|
|            211.75|
|        210.299994|
|212.79999700000002|
|209.18999499999998|
|        207.870005|
|210.11000299999998|
|210.92999500000002|
|        208.330002|
|        214.910006|
|        212.079994|
|206.78000600000001|
|202.51000200000001|
|205.95000100000001|
|        206.849995|
|        204.930004|
|        201.079996|
|192.36999699999998|
+------------------+
only showing top 20 rows



Or both `Open` and `Close` columns:

In [9]:
df.filter("Close < 500").select(['Open', 'Close']).show()

+------------------+------------------+
|              Open|             Close|
+------------------+------------------+
|        213.429998|        214.009998|
|        214.599998|        214.379993|
|        214.379993|        210.969995|
|            211.75|            210.58|
|        210.299994|211.98000499999998|
|212.79999700000002|210.11000299999998|
|209.18999499999998|        207.720001|
|        207.870005|        210.650002|
|210.11000299999998|            209.43|
|210.92999500000002|            205.93|
|        208.330002|        215.039995|
|        214.910006|            211.73|
|        212.079994|        208.069996|
|206.78000600000001|            197.75|
|202.51000200000001|        203.070002|
|205.95000100000001|        205.940001|
|        206.849995|        207.880005|
|        204.930004|        199.289995|
|        201.079996|        192.060003|
|192.36999699999998|        194.729998|
+------------------+------------------+
only showing top 20 rows



## Filtering based on multiple conditions:

In [10]:
df.filter((df['Open'] < 200 ) & (df['Close'] > 200)).show()

+--------------------+----------+------------------+----------+------------------+---------+------------------+
|                Date|      Open|              High|       Low|             Close|   Volume|         Adj Close|
+--------------------+----------+------------------+----------+------------------+---------+------------------+
|2010-02-12 00:00:...|198.109995|        201.639996|195.500002|200.37999299999998|163867200|25.961142000000002|
|2010-02-24 00:00:...|198.229998|201.44000400000002|197.840002|            200.66|115141600|         25.997419|
|2010-02-25 00:00:...|197.380005|        202.859997|196.889994|        202.000004|166281500|          26.17103|
+--------------------+----------+------------------+----------+------------------+---------+------------------+



Showing the result in this format does not look very nice, you can collect the data and save in a variable, so you can work on it later in life.

In [11]:
results = df.filter((df['Open'] < 200)&(df['Close'] > 200)).collect()

It returns a list of row object:

In [12]:
results

[Row(Date=datetime.datetime(2010, 2, 12, 0, 0), Open=198.109995, High=201.639996, Low=195.500002, Close=200.37999299999998, Volume=163867200, Adj Close=25.961142000000002),
 Row(Date=datetime.datetime(2010, 2, 24, 0, 0), Open=198.229998, High=201.44000400000002, Low=197.840002, Close=200.66, Volume=115141600, Adj Close=25.997419),
 Row(Date=datetime.datetime(2010, 2, 25, 0, 0), Open=197.380005, High=202.859997, Low=196.889994, Close=202.000004, Volume=166281500, Adj Close=26.17103)]

In [13]:
results[0]

Row(Date=datetime.datetime(2010, 2, 12, 0, 0), Open=198.109995, High=201.639996, Low=195.500002, Close=200.37999299999998, Volume=163867200, Adj Close=25.961142000000002)

In [14]:
results[0].asDict()

{'Adj Close': 25.961142000000002,
 'Close': 200.37999299999998,
 'Date': datetime.datetime(2010, 2, 12, 0, 0),
 'High': 201.639996,
 'Low': 195.500002,
 'Open': 198.109995,
 'Volume': 163867200}